# Tahoe Forest Health Threshold Analysis
* Mason Bindl, mbindl@trpa.gov
* Andrew McClary, amcclary@trpa.gov

### Setup

In [15]:
# import packages
from utils import *
import arcpy
from arcpy.sa import *
from arcpy.ia import *
from arcpy.ddd import *
import pandas as pd
from pathlib import Path
# setup workspace variables
arcpy.env.workspace = r"F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\ForestHealth_ThresholdUpdate.gdb"
arcpy.env.overwriteOutput = True
#format paths
workspace = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb")
downloads = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\SNV_RRK")
output    = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct")

In [15]:
# data variables
reference_sites     = workspace / "ReferenceSites_ACCEL_30m_SierraNevada"
climate_class_snv   = workspace / "ClimateClasses_15class_ACCEL_30m_SierraNevada"
climate_class_tahoe = workspace / "ClimateClass_Tahoe"

# raw raster data
stand_density       = downloads / "TPA_2021_30m\TPA_2021_30m.tif"
seral_stage         = downloads / "SeralStage_EML_2021\SeralStage_EML_2021.tif"
canopy_cover        = downloads / "CFO_CanopyCover2020Summer\CFO_CanopyCover2020Summer.tif"
fractal_index       = downloads / "FractalIndex_2021\FractalIndex_2021.tif"
fire_prob_low       = downloads / "ProbabilityLowFireSev_202308\ProbabilityLowFireSev_202308.tif"
fire_prob_moderate  = downloads / "ProbabilityMedFireSev_202308\ProbabilityModerateFireSev_202308.tif"
fire_prob_high      = downloads / "ProbabilityHighFireSev_202308\ProbabilityHighFireSev_202308.tif"

### Raster Processing

#### Preprocessing Methods
* Extract existing condition and climate class rasters to Tahoe extent, save in project file geodatabase
* Convert data to single band integer or float rasters, save in the project file geodatabase
* Combine reference sites and climate class


In [ ]:
# process climate class data
# resample to 30m cell size
arcpy.management.Resample(
    in_raster=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\kmeans_15cl_diag_rscl_smooth.tif",
    out_raster="ClimateClasses_15class_ACCEL_30m_SierraNevada",
    cell_size="30 30",
    resampling_type="NEAREST"
)

# extract by mask to Tahoe
climateClassTahoe = ExtractByMask(
    in_raster="ClimateClasses_15class_ACCEL_30m_SierraNevada",
    in_mask_data=r"F:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)
# save tahoe climate class raster
climateClassTahoe.save("ClimateClass_Tahoe")

In [ ]:
# process reference sites
Int(
    in_raster_or_constant=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\referenceYesNo_3crit.tif",
    out_raster="referencesites_int"
)

arcpy.conversion.PolygonToRaster(
    in_features="F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\contemporary_reference_sites.shp",
    value_field="site_id",
    out_rasterdataset="referencesites_contemporary_int",
    cell_assignment="CELL_CENTER",
    priority_field="NONE",
    cellsize=30,
    build_rat="BUILD"
)

arcpy.management.MosaicToNewRaster(
    input_rasters="referencesites_contemporary_int;referencesites_int",
    output_location=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb",
    raster_dataset_name_with_extension="Mosaic_ReferenceSites_ACCEL_30m_SierraNevada",
    coordinate_system_for_the_raster='PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    pixel_type="8_BIT_UNSIGNED",
    cellsize=30,
    number_of_bands=1,
    mosaic_method="LAST",
    mosaic_colormap_mode="FIRST"
)

Reclassify(
    in_raster="Mosaic_ReferenceSites_ACCEL_30m_SierraNevada",
    reclass_field="Value",
    remap="1 119 1",
    out_raster = "ReferenceSites_ACCEL_30m_SierraNevada",
    missing_values="DATA"
)

ExtractClimateClassesbyReferenceSites = ExtractByMask(
    in_raster="ClimateClasses_15class_ACCEL_30m_SierraNevada",
    in_mask_data="ReferenceSites_ACCEL_30m_SierraNevada",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)
ExtractClimateClassesbyReferenceSites.save("ExtractByMask_ClimateClasses_ReferenceSites_ACCEL_30m_SierraNevada")

#### Process Climate Classes

In [ ]:
# get field names to loop through and create rasters from
field_names = [f.name for f in arcpy.ListFields("ClimateClass_Tahoe")]
print(field_names)

In [ ]:
fcList = ["ClimateClass_Tahoe"]
for fc in fcList:
    fields_to_delete = [field.name for field in arcpy.ListFields(fc) if not field.required]
#     fields_to_delete.pop() #Keep one non-required field
    for field in fields_to_delete:
        arcpy.DeleteField_management(fc, field)

In [ ]:
keep_fields = ["",""]
fieldNameList = []
fieldObjList = arcpy.ListFields(table)

for field in fieldObjList:
    if (not field.name in keep_fields) and (not field.required):
        fieldNameList.append(field.name)

arcpy.DeleteField_management(table,fieldNameList)

In [ ]:
add_acres("ClimateClass_Tahoe")

In [ ]:
arcpy.conversion.ExportTable(
    in_table=r"Boundaries\Climate Classes - Tahoe",
    out_table=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\ClimateClasses_TahoeValues.csv",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping=r'Value "Value" false true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,Value,-1,-1;Count "Count" false true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,Count,-1,-1;StandDensity10thPercentile "StandDensity10thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,StandDensity10thPercentile,-1,-1;StandDensity90thPercentile "StandDensity90thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,StandDensity90thPercentile,-1,-1;LargeTreeDensity10thPercentile "LargeTreeDensity10thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,LargeTreeDensity10thPercentile,-1,-1;LargeTreeDensity20thPercentile "LargeTreeDensity20thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,LargeTreeDensity20thPercentile,-1,-1;LargeTreeDensity80thPercentile "LargeTreeDensity80thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,LargeTreeDensity80thPercentile,-1,-1;LargeTreeDensity90thPercentile "LargeTreeDensity90thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,LargeTreeDensity90thPercentile,-1,-1;FractalIndex10thPercentile "FractalIndex10thPercentile" true true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,FractalIndex10thPercentile,-1,-1;FractalIndex20thPercentile "FractalIndex20thPercentile" true true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,FractalIndex20thPercentile,-1,-1;FractalIndex80thPercentile "FractalIndex80thPercentile" true true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,FractalIndex80thPercentile,-1,-1;FractalIndex90thPercentile "FractalIndex90thPercentile" true true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,FractalIndex90thPercentile,-1,-1;Acres "Acres" true true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,Acres,-1,-1',
    sort_field=None
)

## Stand Density

### Methods
* Extracted climate classes by contemporary reference sites to use as the zonal input to calculate 10th and 90th percentile of Trees Per Acres
* Extract by mask Tahoe Climate Classes and Tahoe Trees Per Acre values
* Classify new rasters of 10th and 90th percentile for Tahoe by climate class
* Compare current Tahoe Trees Per Acre pixel values to 10th and 90th percentile climate class/reference site values and classify whether the pixel is below target (<10th percentile), at target (between 10th and 90th percentile, or above target (>90th percentile)

In [ ]:
# input Tahoe ACCEL Climate Class Raster
climateClass = "ClimateClass_Tahoe"

# input zones
zones        = "ExtractByMask_ClimateClasses_ReferenceSites_ACCEL_30m_SierraNevada"

# output zonal stats table
zonalStats   = "ZonalStats_ReferenceSiteClimateClass_StandDensity_Percentile"

# out field names
field10th    = 'StandDensity10thPercentile'
field90th    = 'StandDensity90thPercentile'

# name the output climate class/reference site metric stat rasters
name10th     = os.path.join(workspace, f"ClimateClassTahoe_{field10th}")
name90th     = os.path.join(workspace, f"ClimateClassTahoe_{field90th}")

# declare rasters to use as inputs
standDensitySierra = Raster("StandDensity_TPA_ACCEL_30m_SierraNevada")
standDensityTahoe  = Raster("StandDensity_TPA_ACCEL_30m_Tahoe")
standDensity10th   = Raster(name10th)
standDensity90th   = Raster(name90th)

# name output classified raster
standDensityName   = "StandDensity_TPA_Classified_30m_Tahoe_9010"

# run zonal stats
ZonalStatisticsAsTable(
    in_zone_data=zones,
    zone_field="Value",
    in_value_raster=standDensitySierra,
    out_table=zonalStats,
    ignore_nodata="DATA",
    statistics_type="PERCENTILE",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[90,10],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field10th])

# add field
arcpy.management.AddField(climateClass, 
                          field10th, 
                          "LONG"
                         )

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field90th])

# add field
arcpy.management.AddField(climateClass, 
                          field90th, 
                          "LONG"
                         )


# add join to zonal stats table
arcpy.management.AddJoin(
    in_layer_or_view= climateClass,
    in_field= "Value",
    join_table= zonalStats,
    join_field= "Value",
    join_type= "KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# calc fields
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field90th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_StandDensity_Percentile.PCT90!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field10th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_StandDensity_Percentile.PCT10!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)

# remove the join
arcpy.RemoveJoin_management("ClimateClass_Tahoe_Layer", 
                            zonalStats)

# save rasters for lookup
Lookup(
    in_raster=climateClass,
    lookup_field=field10th,
    out_raster=name10th
)
print(f"Exported raster for {field10th} and saved it here {name10th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field90th,
    out_raster=name90th
)
print(f"Exported raster for {field90th} and saved it here {name90th}")


# Use the Con function to classify the raster based on reference condition range values
# -1 represents less than 10th percentile, 1 is greater than 90th, 0 is target
standDensityClass = Con(standDensityTahoe < standDensity10th, -1,
                    Con(standDensityTahoe > standDensity90th, 1, 0))

# Save the output raster to the geodatabase
standDensityClass.save(standDensityName)
print(f"Exported raster: {standDensityName}")

# lookup value category
lookup_dict = {-1:"Below Target", 0:"Target", 1:"Above Target"}
# build an attribute table
arcpy.BuildRasterAttributeTable_management(standDensityName, "Overwrite")
# calc Acres field
add_acres(standDensityName)
# add and calc category field
add_category(standDensityName, lookup_dict)

In [ ]:
# input Tahoe ACCEL Climate Class Raster
climateClass = "ClimateClass_Tahoe"

# input zones
zones        = "ExtractByMask_ClimateClasses_ReferenceSites_ACCEL_30m_SierraNevada"

# output zonal stats table
zonalStats   = "ZonalStats_ReferenceSiteClimateClass_StandDensity_Percentile"

# out field names
field30th    = 'StandDensity30thPercentile'
field70th    = 'StandDensity70thPercentile'

# name the output climate class/reference site metric stat rasters
name30th     = os.path.join(workspace, f"ClimateClassTahoe_{field30th}")
name70th     = os.path.join(workspace, f"ClimateClassTahoe_{field70th}")

# declare rasters to use as inputs
standDensitySierra = Raster("StandDensity_TPA_ACCEL_30m_SierraNevada")
standDensityTahoe  = Raster("StandDensity_TPA_ACCEL_30m_Tahoe")
standDensity30th   = Raster(name30th)
standDensity70th   = Raster(name70th)

# name output classified raster
standDensityName   = "StandDensity_TPA_Classified_30m_Tahoe_7030"

# run zonal stats
ZonalStatisticsAsTable(
    in_zone_data=zones,
    zone_field="Value",
    in_value_raster=standDensitySierra,
    out_table=zonalStats,
    ignore_nodata="DATA",
    statistics_type="PERCENTILE",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[90,80,70,60,50,40,30,20,10],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field30th])

# add field
arcpy.management.AddField(climateClass, 
                          field30th, 
                          "LONG"
                         )

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field70th])

# add field
arcpy.management.AddField(climateClass, 
                          field70th, 
                          "LONG"
                         )


# add join to zonal stats table
arcpy.management.AddJoin(
    in_layer_or_view= climateClass,
    in_field= "Value",
    join_table= zonalStats,
    join_field= "Value",
    join_type= "KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# calc fields
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field70th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_StandDensity_Percentile.PCT70!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field30th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_StandDensity_Percentile.PCT30!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)

# remove the join
arcpy.RemoveJoin_management("ClimateClass_Tahoe_Layer", 
                            zonalStats)

# save rasters for lookup
Lookup(
    in_raster=climateClass,
    lookup_field=field30th,
    out_raster=name10th
)
print(f"Exported raster for {field30th} and saved it here {name30th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field70th,
    out_raster=name90th
)
print(f"Exported raster for {field70th} and saved it here {name70th}")


# Use the Con function to classify the raster based on reference condition range values
# -1 represents less than 10th percentile, 1 is greater than 90th, 0 is target
standDensityClass = Con(standDensityTahoe < standDensity30th, -1,
                    Con(standDensityTahoe > standDensity70th, 1, 0))

# Save the output raster to the geodatabase
standDensityClass.save(standDensityName)
print(f"Exported raster: {standDensityName}")

# lookup value category
lookup_dict = {-1:"Below Target", 0:"Target", 1:"Above Target"}
# build an attribute table
arcpy.BuildRasterAttributeTable_management(standDensityName, "Overwrite")
# calc Acres field
add_acres(standDensityName)
# add and calc category field
add_category(standDensityName, lookup_dict)

In [ ]:
arcpy.conversion.ExportTable(
    in_table="StandDensity_TPA_Classified_30m_Tahoe",
    out_table=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\StandDensity_TPA_Classified_30m_Tahoe.csv",
)


In [ ]:
dfTPA = pd.read_csv(r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\StandDensity_TPA_Classified_30m_Tahoe.csv")

In [ ]:
dfTPA

## Large Tree Density

### Methods
* Extracted climate classes by contemporary reference sites to use as the zonal input to calculate 10th and 90th percentile of number of Trees Per Acre >30in DBH
* Extract by mask Tahoe Climate Classes and Tahoe number of Trees Per Acre >30in DBH values
* Classify new rasters of 10th and 90th percentile for Tahoe by climate class
* Compare current Tahoe number of Trees Per Acre >30in DBH pixel values to 10th and 90th percentile climate class/reference site values and classify whether the pixel is below target (<10th percentile), at target (between 10th and 90th percentile, or above target (>90th percentile)

In [ ]:
# save integer version of large tree density and extract by mask to Tahoe
Int(
    in_raster_or_constant="F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\Download\\ACCEL\\DensityLargeTrees\\TPA_30in_up_2021_30m.tif",
    out_raster="LargeTreeDensity_TPA30inUp_ACCEL_30m_SierraNevada"
)
# extract by mask to Tahoe extent
out_raster = ExtractByMask(
    in_raster="LargeTreeDensity_TPA30inUp_ACCEL_30m_SierraNevada",
    in_mask_data=r"F:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)
# save.
out_raster.save("LargeTreeDensity_TPA30inUp_ACCEL_30m_Tahoe")

In [ ]:
# input Tahoe ACCEL Climate Class Raster
climateClass = "ClimateClass_Tahoe"

# input zones
zones        = "ExtractByMask_ClimateClasses_ReferenceSites_ACCEL_30m_SierraNevada"

# output zonal stats table
zonalStats   = "ZonalStats_ReferenceSiteClimateClass_LargeTreeDensity_Percentile"

# out field names
field10th    = 'LargeTreeDensity10thPercentile'
field20th    = 'LargeTreeDensity20thPercentile'
field80th    = 'LargeTreeDensity80thPercentile'
field90th    = 'LargeTreeDensity90thPercentile'

# name the output climate class/reference site metric stat rasters
name10th     = os.path.join(workspace, f"ClimateClassTahoe_{field10th}")
name20th     = os.path.join(workspace, f"ClimateClassTahoe_{field20th}")
name80th     = os.path.join(workspace, f"ClimateClassTahoe_{field80th}")
name90th     = os.path.join(workspace, f"ClimateClassTahoe_{field90th}")

# lookup value category
lookup_dict = {-1:"Below Target", 0:"Target", 1:"Above Target"}

# declare rasters to use as inputs
largetreeDensitySierra = Raster("LargeTreeDensity_TPA30inUp_ACCEL_30m_SierraNevada")
largetreeDensityTahoe  = Raster("LargeTreeDensity_TPA30inUp_ACCEL_30m_Tahoe")

# name output classified raster
largetreeDensityName1090   = "LargeTreeDensity_TPA_Classified_30m_Tahoe_1090"

# name output classified raster
largetreeDensityName2080   = "LargeTreeDensity_TPA_Classified_30m_Tahoe_2080"

# run zonal stats
ZonalStatisticsAsTable(
    in_zone_data=zones,
    zone_field="Value",
    in_value_raster=largetreeDensitySierra,
    out_table=zonalStats,
    ignore_nodata="DATA",
    statistics_type="PERCENTILE",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[10, 20, 80, 90],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field10th])

# add field
arcpy.management.AddField(climateClass, 
                          field10th, 
                          "LONG"
                         )

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field20th])

# add field
arcpy.management.AddField(climateClass, 
                          field20th, 
                          "LONG"
                         )
# delete field
arcpy.DeleteField_management(climateClass, 
                             [field80th])

# add field
arcpy.management.AddField(climateClass, 
                          field80th, 
                          "LONG"
                         )

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field90th])

# add field
arcpy.management.AddField(climateClass, 
                          field90th, 
                          "LONG"
                         )


# add join to zonal stats table
arcpy.management.AddJoin(
    in_layer_or_view= climateClass,
    in_field= "Value",
    join_table= zonalStats,
    join_field= "Value",
    join_type= "KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# calc fields
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field10th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_LargeTreeDensity_Percentile.PCT10!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field20th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_LargeTreeDensity_Percentile.PCT20!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field80th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_LargeTreeDensity_Percentile.PCT80!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field90th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_LargeTreeDensity_Percentile.PCT90!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)

# remove the join
arcpy.RemoveJoin_management("ClimateClass_Tahoe_Layer", zonalStats)

# save rasters for lookup
Lookup(
    in_raster=climateClass,
    lookup_field=field10th,
    out_raster=name10th
)
print(f"Exported raster for {field10th} and saved it here {name10th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field20th,
    out_raster=name20th
)
print(f"Exported raster for {field20th} and saved it here {name20th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field80th,
    out_raster=name80th
)
print(f"Exported raster for {field80th} and saved it here {name80th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field90th,
    out_raster=name90th
)
print(f"Exported raster for {field90th} and saved it here {name90th}")

largetreeDensity10th   = Raster(name10th)
largetreeDensity20th   = Raster(name20th)
largetreeDensity80th   = Raster(name80th)
largetreeDensity90th   = Raster(name90th)

# Use the Con function to classify the raster based on reference condition range values
# -1 represents less than 10th percentile, 1 is greater than 90th, 0 is target
largetreeDensityClass1090 = Con(largetreeDensityTahoe < largetreeDensity10th, -1,
                            Con(largetreeDensityTahoe > largetreeDensity90th, 1, 0))

# Save the output raster to the geodatabase
largetreeDensityClass1090.save(largetreeDensityName1090)
print(f"Exported raster: {largetreeDensityName1090}")

# build an attribute table
arcpy.BuildRasterAttributeTable_management(largetreeDensityName1090, "Overwrite")
# calc Acres field
add_acres(largetreeDensityName1090)
# add and calc category field
add_category(largetreeDensityName1090, lookup_dict)


# Use the Con function to classify the raster based on reference condition range values
# -1 represents less than 10th percentile, 1 is greater than 90th, 0 is target
largetreeDensityClass2080 = Con(largetreeDensityTahoe < largetreeDensity20th, -1,
                            Con(largetreeDensityTahoe > largetreeDensity80th, 1, 0))

# Save the output raster to the geodatabase
largetreeDensityClass2080.save(largetreeDensityName2080)
print(f"Exported raster: {largetreeDensityName2080}")

# build an attribute table
arcpy.BuildRasterAttributeTable_management(largetreeDensityName2080, "Overwrite")
# calc Acres field
add_acres(largetreeDensityName2080)
# add and calc category field
add_category(largetreeDensityName2080, lookup_dict)

In [ ]:
arcpy.conversion.ExportTable(
    in_table="LargeTreeDensity_TPA_Classified_30m_Tahoe_1090",
    out_table=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\LargeTreeDensity_Class_10th90thPercentile.csv",
)
arcpy.conversion.ExportTable(
    in_table="LargeTreeDensity_TPA_Classified_30m_Tahoe_2080",
    out_table=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\LargeTreeDensity_Class_20th80thPercentile.csv",
)

In [ ]:
df10th90th = pd.read_csv(r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\LargeTreeDensity_Class_10th90thPercentile.csv")
df20th80th = pd.read_csv(r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\LargeTreeDensity_Class_20th80thPercentile.csv")

In [ ]:
df20th80th

In [ ]:
df10th90th

## Horizontal Heterogeneity

In [ ]:
# save integer version of large tree density and extract by mask to Tahoe
Float(
    in_raster_or_constant="F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\Download\\ACCEL\\FSHFineScaleHeterogeneityIndex\\fractal_dim_spring_2020_30m.tif",
    out_raster="HorizontalHeterogeneity_FractalIndex_ACCEL_30m_SierraNevada"
)
# extract by mask to Tahoe extent
out_raster = ExtractByMask(
    in_raster="HorizontalHeterogeneity_FractalIndex_ACCEL_30m_SierraNevada",
    in_mask_data=r"F:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)
# save.
out_raster.save("HorizontalHeterogeneity_FractalIndex_ACCEL_30m_Tahoe")

In [ ]:
# input Tahoe ACCEL Climate Class Raster
climateClass = "ClimateClass_Tahoe"

# input zones
zones        = "ExtractByMask_ClimateClasses_ReferenceSites_ACCEL_30m_SierraNevada"

# output zonal stats table
zonalStats   = "ZonalStats_ReferenceSiteClimateClass_FractalIndex_Percentile"

# out field names
field10th    = 'FractalIndex10thPercentile'
field20th    = 'FractalIndex20thPercentile'
field80th    = 'FractalIndex80thPercentile'
field90th    = 'FractalIndex90thPercentile'

# name the output climate class/reference site metric stat rasters
name10th     = os.path.join(workspace, f"ClimateClassTahoe_{field10th}")
name20th     = os.path.join(workspace, f"ClimateClassTahoe_{field20th}")
name80th     = os.path.join(workspace, f"ClimateClassTahoe_{field80th}")
name90th     = os.path.join(workspace, f"ClimateClassTahoe_{field90th}")

# lookup value category
lookup_dict = {-1:"Below Target", 0:"Target", 1:"Above Target"}

# declare rasters to use as inputs
fractalIndexSierra = Raster("HorizontalHeterogeneity_FractalIndex_ACCEL_30m_SierraNevada")
fractalIndexTahoe  = Raster("HorizontalHeterogeneity_FractalIndex_ACCEL_30m_Tahoe")

# name output classified raster
fractalIndexName1090   = "FractalIndex_Classified_Tahoe_1090"

# name output classified raster
fractalIndexName2080   = "FractalIndex_Classified_Tahoe_2080"

# run zonal stats
ZonalStatisticsAsTable(
    in_zone_data=zones,
    zone_field="Value",
    in_value_raster=fractalIndexSierra,
    out_table=zonalStats,
    ignore_nodata="DATA",
    statistics_type="PERCENTILE",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[10, 20, 80, 90],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field10th])

# add field
arcpy.management.AddField(climateClass, 
                          field10th, 
                          "DOUBLE"
                         )

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field20th])

# add field
arcpy.management.AddField(climateClass, 
                          field20th, 
                          "DOUBLE"
                         )
# delete field
arcpy.DeleteField_management(climateClass, 
                             [field80th])

# add field
arcpy.management.AddField(climateClass, 
                          field80th, 
                          "DOUBLE"
                         )

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field90th])

# add field
arcpy.management.AddField(climateClass, 
                          field90th, 
                          "DOUBLE"
                         )


# add join to zonal stats table
arcpy.management.AddJoin(
    in_layer_or_view= climateClass,
    in_field= "Value",
    join_table= zonalStats,
    join_field= "Value",
    join_type= "KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# calc fields
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field10th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_FractalIndex_Percentile.PCT10!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field20th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_FractalIndex_Percentile.PCT20!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field80th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_FractalIndex_Percentile.PCT80!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field90th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_FractalIndex_Percentile.PCT90!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE"
)

# remove the join
arcpy.RemoveJoin_management("ClimateClass_Tahoe_Layer", zonalStats)

# save rasters for lookup
Lookup(
    in_raster=climateClass,
    lookup_field=field10th,
    out_raster=name10th
)
print(f"Exported raster for {field10th} and saved it here {name10th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field20th,
    out_raster=name20th
)
print(f"Exported raster for {field20th} and saved it here {name20th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field80th,
    out_raster=name80th
)
print(f"Exported raster for {field80th} and saved it here {name80th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field90th,
    out_raster=name90th
)
print(f"Exported raster for {field90th} and saved it here {name90th}")

fractalIndex10th   = Raster(name10th)
fractalIndex20th   = Raster(name20th)
fractalIndex80th   = Raster(name80th)
fractalIndex90th   = Raster(name90th)

# Use the Con function to classify the raster based on reference condition range values
# -1 represents less than 10th percentile, 1 is greater than 90th, 0 is target
fractalIndexClass1090 = Con(fractalIndexTahoe < fractalIndex10th, -1,
                            Con(fractalIndexTahoe > fractalIndex90th, 1, 0))

# Save the output raster to the geodatabase
fractalIndexClass1090.save(fractalIndexName1090)
print(f"Exported raster: {largetreeDensityName1090}")

# build an attribute table
arcpy.BuildRasterAttributeTable_management(fractalIndexName1090, "Overwrite")
# calc Acres field
add_acres(fractalIndexName1090)
# add and calc category field
add_category(fractalIndexName1090, lookup_dict)


# Use the Con function to classify the raster based on reference condition range values
# -1 represents less than 10th percentile, 1 is greater than 90th, 0 is target
fractalIndexClass2080 = Con(fractalIndexTahoe < fractalIndex20th, -1,
                            Con(fractalIndexTahoe > fractalIndex80th, 1, 0))

# Save the output raster to the geodatabase
fractalIndexClass2080.save(fractalIndexName2080)
print(f"Exported raster: {fractalIndexName2080}")

# build an attribute table
arcpy.BuildRasterAttributeTable_management(fractalIndexName2080, "Overwrite")
# calc Acres field
add_acres(fractalIndexName2080)
# add and calc category field
add_category(fractalIndexName2080, lookup_dict)

In [ ]:
arcpy.conversion.ExportTable(
    in_table="FractalIndex_Classified_Tahoe_1090",
    out_table=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\FractalIndex_Class_10th90thPercentile.csv",
)
arcpy.conversion.ExportTable(
    in_table="FractalIndex_Classified_Tahoe_2080",
    out_table=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\FractalIndex_Class_20th80thPercentile.csv",
)

In [ ]:
df1090 = pd.read_csv("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\FractalIndex_Class_10th90thPercentile.csv")
df2080 = pd.read_csv("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\FractalIndex_Class_20th80thPercentile.csv")

## Seral Stage + Canopy Cover

In [ ]:
seral_stage         = downloads / "SeralStage_EML_2021\SeralStage_EML_2021.tif"
canopy_cover        = downloads / "CFO_CanopyCover2020Summer\CFO_CanopyCover2020Summer.tif"

Seral   = Raster(str(seral_stage))
Canopy  = Raster(str(canopy_cover))

# dictionay of lookup values
lookup_dict = {1:"Early Seral Open", 
               2:"Early Seral Closed", 
               3:"Mid Seral Open", 
               4:"Mid Seral Closed", 
               5:"Late Seral Open", 
               6:"Late Seral Closed"}

# Use the Con function to set the output values 1=Early Seral Open, etc..
SeralCanopy40 =         Con(((Raster(Seral) == 1) & (Raster(Canopy) <40)),1, 
                        Con(((Raster(Seral) == 1) & (Raster(Canopy)>=40)),2,
                        Con(((Raster(Seral) == 2) & (Raster(Canopy) <40)),3, 
                        Con(((Raster(Seral) == 2) & (Raster(Canopy)>=40)),4,
                        Con(((Raster(Seral) == 3) & (Raster(Canopy) <40)),5, 
                        Con(((Raster(Seral) == 3) & (Raster(Canopy)>=40)),6))))))

# Use the Con function to set the output values 
SeralCanopy60 =         Con(((Raster(Seral) == 1) & (Raster(Canopy) <60)),1, 
                        Con(((Raster(Seral) == 1) & (Raster(Canopy)>=60)),2,
                        Con(((Raster(Seral) == 2) & (Raster(Canopy) <60)),3, 
                        Con(((Raster(Seral) == 2) & (Raster(Canopy)>=60)),4,
                        Con(((Raster(Seral) == 3) & (Raster(Canopy) <60)),5, 
                        Con(((Raster(Seral) == 3) & (Raster(Canopy)>=60)),6))))))

# Set output raster
output_raster40 = "SeralStage_CanopyClass_40prcnt_30m_Tahoe_SNVRRK"
output_raster60 = "SeralStage_CanopyClass_60prcnt_30m_Tahoe_SNVRRK"
# Save the output raster to the geodatabase
SeralCanopy40.save(output_raster40)
SeralCanopy60.save(output_raster60)

# build an attribute table
arcpy.BuildRasterAttributeTable_management(output_raster40, "Overwrite")
arcpy.BuildRasterAttributeTable_management(output_raster60, "Overwrite")
# calc Acres field
add_acres(output_raster40)
add_acres(output_raster60)
# add and calc category field
add_category(output_raster40, lookup_dict)
add_category(output_raster60, lookup_dict)

In [12]:

# get raster attribute table as a pandas dataframe
arcpy.conversion.ExportTable(
    in_table=output_raster40,
    out_table= str(output / "SeralStage_CanopyClass_40prcnt_30m_Tahoe_SNVRRK.csv")
)
arcpy.conversion.ExportTable(
    in_table=output_raster60,
    out_table= str(output / "SeralStage_CanopyClass_60prcnt_30m_Tahoe_SNVRRK.csv")
)
# read the csv
df40 = pd.read_csv(output / "SeralStage_CanopyClass_40prcnt_30m_Tahoe_SNVRRK.csv")
df60 = pd.read_csv(output / "SeralStage_CanopyClass_60prcnt_30m_Tahoe_SNVRRK.csv")

In [14]:
df40

,Value,Count,Acres,Category
0,1,4406868.0,9.800654e+05,Early Seral Open
1,2,3120970.0,6.940881e+05,Early Seral Closed
2,3,22271644.0,4.953102e+06,Mid Seral Open
3,4,23708468.0,5.272645e+06,Mid Seral Closed
4,5,5123573.0,1.139457e+06,Late Seral Open
5,6,8204643.0,1.824672e+06,Late Seral Closed


## Functional Fire

In [ ]:
# Reclassify based on 60% cutoff
outReclass = Reclassify("FunctionalFire_ProbHighSeverity_2022_ACCEL_30m_Tahoe", "Value", 
                        RemapRange([[0,0.6,0],[0.6,1,1]])) 
# save raster
outReclass.save("FunctionalFire_Reclass_High_60thPercentile")

In [ ]:
# reclassify based on 60 percent cutoff
outReclass = Reclassify("FunctionalFire_ProbLowSeverity_2022_ACCEL_30m_Tahoe", "Value", 
                        RemapRange([[0,0.6,0],[0.6,1,1]])) 
# save raster
outReclass.save("FunctionalFire_Reclass_Low_60thPercentile")

In [ ]:
attribute_tables=["FunctionalFire_Reclass_Low_60thPercentile", "FunctionalFire_Reclass_High_60thPercentile", "heterogeneity_category_fractal", "heterogeneity_category_canopycover"]  
lookup_dict - {-1:"Lower", 0:"Target", 1:"Higher"}

for attribute_table in attribute_tables:
    add_acres(attribute_table)
    add_category(attribute_table, lookup_dict)
    